In [40]:
import pandas as pd
import torch
from torch import nn
from ipdb import set_trace
import torchvision as tv
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader as DL
from utils.OID_Dataset import OID_Dataset as DS

### load data

In [41]:
__ds_root__ = '../google_oid_v6'
__train_f__ = 'oidv6-train-annotations-bbox.csv'
__test_f__ = '../google_oid_v6/ANNOT/test-annotations-bbox.csv'
__val_f__ = '../google_oid_v6/ANNOT/validation-annotations-bbox.csv'

ind = 0

classes = set(('Person', 'Car', 'Mobile phone'))
labels = pd.read_csv(f'{__ds_root__}/ANNOT/class-descriptions-boxable.csv')
label_ids = set(labels[labels.iloc[:,1].isin(classes)].iloc[:, 0])

In [45]:
# use custom collate, variable # of boxes in each image
def var_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    
    return torch.stack(data, dim=0), target

oid_ds = DS(__ds_root__, __train_f__, label_ids)
train_load = DL(dataset=oid_ds, batch_size=4, shuffle=True, num_workers=4, collate_fn=var_collate)

In [67]:
class VGG(nn.Module):
    def __init__(self, c=64):
        super(VGG, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, c, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c, c, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(c, c*2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*2, c*2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(c*2, c*4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*4, c*4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*4, c*4, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(c*4, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(c*8, c*8, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, padding=0),
        )
        
        self.conn = nn.Sequential(
            nn.Linear(7 * 7 * 512, 4096),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(),
            nn.Linear(4096, 1470),
        )
        
        
    def forward(self, img):
        # conv layers
        h = self.conv1(img)
        h = self.conv2(h)
        h = self.conv3(h)
        h = self.conv4(h)
        h = self.conv5(h)
        # output through FC
        h = h.reshape(h.shape[0], -1)
        h = self.conn(h)
        h = h.reshape(h.shape[0], 30, 7, 7)
        
        return h
    
class RPN(nn.Module):
    def __init__(self, c=512, ratio=[1.0, 2.0, 0.5]):
        super(RPN, self).__init__()
#         self.feat = nn.Conv2d(c, c/2, 3, stride=1, padding=1)
#         self.objness = nn.Conv2d()
    def forward(self, feat):
        pass

In [68]:
conv_net = VGG().cuda(ind)
yolo_net = YOLO_FC().cuda(ind)

for img, y in train_load:
    img = img.cuda(ind)
    feat = conv_net(img)
    set_trace()


> <ipython-input-68-976b5a750cad>(4)<module>()
      3 
----> 4 for img, y in train_load:
      5     img = img.cuda(ind)

ipdb> feat.shape
torch.Size([4, 30, 7, 7])
ipdb> exit


BdbQuit: 

In [53]:
%debug

> /usr/local/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py(1354)linear()
   1352         ret = torch.addmm(torch.jit._unwrap_optional(bias), input, weight.t())
   1353     else:
-> 1354         output = input.matmul(weight.t())
   1355         if bias is not None:
   1356             output += torch.jit._unwrap_optional(bias)

ipdb> u
> /usr/local/miniconda3/lib/python3.7/site-packages/torch/nn/modules/linear.py(67)forward()
     65     @weak_script_method
     66     def forward(self, input):
---> 67         return F.linear(input, self.weight, self.bias)
     68 
     69     def extra_repr(self):

ipdb> self.weight.shape
torch.Size([4096, 25088])
ipdb> exit


[(0, 'Car'), (1, 'Person'), (2, 'Mobile phone')]
